In [1]:
import sparknlp

spark = sparknlp.start()

24/11/13 20:30:51 WARN Utils: Your hostname, ubuntu-IdeaPad-Gaming-3-15IAH7 resolves to a loopback address: 127.0.1.1; using 192.168.1.16 instead (on interface enp48s0)
24/11/13 20:30:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/.local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7dfeb99f-8f9f-49d3-bb93-04a371245a0c;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500 in centra

In [ ]:
from sparknlp.annotator import (
    Tokenizer, BertForZeroShotClassification
    )
from pyspark.ml import Pipeline
from sparknlp.base import DocumentAssembler

document_assembler = DocumentAssembler() \
.setInputCol('text') \
.setOutputCol('document')

tokenizer = Tokenizer() \
.setInputCols(['document']) \
.setOutputCol('token')

zeroShotClassifier = BertForZeroShotClassification \
.pretrained('bert_base_cased_zero_shot_classifier_xnli', 'en') \
.setInputCols(['token', 'document']) \
.setOutputCol('class') \
.setCaseSensitive(True) \
.setMaxSentenceLength(512) \
.setCandidateLabels(["Credit reporting", "Debt collection", "Mortgage", "Student loan", "Credit card", "Checking or savings account", "Money transfer", "Vehicle loan or lease", "Personal loan", "Fraud, theft and scam", "Other"])

pipeline = Pipeline(stages=[
document_assembler,
tokenizer,
zeroShotClassifier
])

bert_base_cased_zero_shot_classifier_xnli download started this may take some time.
Approximate size to download 387.7 MB
[ | ]

24/11/13 20:31:19 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
24/11/13 20:31:19 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


bert_base_cased_zero_shot_classifier_xnli download started this may take some time.
Approximate size to download 387.7 MB
Download done! Loading the resource.
[ \ ]

2024-11-13 20:31:26.830694: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[ | ]

[OK!]


In [4]:
example = spark.createDataFrame([["My credit card was stolen, and I want to file a formal complaint against the Credit Reporting Agencies (CRAs) regarding the inaccurate reporting of my credit information."]]).toDF("text")
from pyspark.sql.functions import substring

# Limitar el texto a los primeros 512 caracteres (aproximadamente)
example = example.withColumn("text", substring("text", 1, 512))
result = pipeline.fit(example).transform(example)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

To Whom It May Concern, Subject : Formal Complaint, Opt-Out of Inaccurate Reporting, and Demand for Immediate Updates I am writing to file a formal complaint against the Credit Reporting Agencies ( CRAs ) regarding the inaccurate reporting of my credit information. As a federally protected consumer under the Fair Credit Reporting Act ( FCRA ), specifically 15 U.S.C. 1681 et seq., I am exercising my rights concerning the privacy of my personal information and the accuracy of my credit report. The FCRA mandat